In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = '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',
    model="llama-3.1-70b-versatile"   
)

In [1]:
from langchain_community.document_loaders import WebBaseLoader

In [2]:
loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)



Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsFIND A STOREBECOME A MEMBERSIGN UP FOR EMAILSEND US FEEDBACKSTUDENT DISCOUNTSGET HELPGET HELPOrder StatusDeliveryReturnsPayment OptionsC

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###INSTRUCTION:
    The scrped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing
    following keys: 'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

In [6]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input = {'page_data': page_data})
print(res.content)

Unfortunately, the provided text does not contain any job postings with the required information. However, I can extract the job categories and provide them in JSON format. Please note that the 'experience' and 'skills' fields will be empty as they are not available in the provided text.

```json
[
  {
    "role": "Administrative Support",
    "experience": "",
    "skills": "",
    "description": ""
  },
  {
    "role": "Advanced Innovation",
    "experience": "",
    "skills": "",
    "description": ""
  },
  {
    "role": "Air Manufacturing Innovation",
    "experience": "",
    "skills": "",
    "description": ""
  },
  {
    "role": "Aviation",
    "experience": "",
    "skills": "",
    "description": ""
  },
  {
    "role": "Communications",
    "experience": "",
    "skills": "",
    "description": ""
  },
  {
    "role": "Customer Service",
    "experience": "",
    "skills": "",
    "description": ""
  },
  {
    "role": "Design",
    "experience": "",
    "skills": "",
    "

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Administrative Support',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Advanced Innovation',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Air Manufacturing Innovation',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Aviation', 'experience': '', 'skills': '', 'description': ''},
 {'role': 'Communications', 'experience': '', 'skills': '', 'description': ''},
 {'role': 'Customer Service',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Design', 'experience': '', 'skills': '', 'description': ''},
 {'role': 'Digital', 'experience': '', 'skills': '', 'description': ''},
 {'role': 'Facilities', 'experience': '', 'skills': '', 'description': ''},
 {'role': 'Finance & Accounting',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Government & Public Affairs',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Human Resources',
  'experience': '',
  'skills': 

In [8]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df.head()


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name = "portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents = row["Techstack"],
                       metadatas = {"links": row["Links"]},
                       ids = [str(uuid.uuid4())])

In [14]:
job = json_res

In [18]:
for job_item in job:
    links = collection.query(query_texts=job_item['skills'], n_results=2).get('metadatas', [])

links

[[{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Natasha Romanoff, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Efficiency and Scalability with AtliQ's AI-Powered Solutions

Dear Hiring Manager,

I came across your job postings and was impressed by the diverse range of roles you're looking to fill. As a Business Development Executive at AtliQ, I'd like to introduce you to our company's capabilities in providing tailored solutions that can empower your organization to achieve scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team of experts has extensive experience in developing and implementing AI-powered solutions that cater to various industries and business needs. We've successfully empowered numerous enterprises with our automated tools, and I'd like to highlight a few examples from our portfolio:

* Our Android TV portfolio showcases our expertise in developing innovative solutions for the entertainment industry. (https://example.com/android-tv-portfolio)
* Our Kotlin backend portfolio demonstrates our proficiency in building robust a